In [3]:
import pandas as pd
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset containing the range information and crop labels
dataset = pd.read_csv('dataset.csv')

# Create combo box for District_name
district_widget = widgets.Combobox(
    options=tuple(dataset['District_Name'].unique()),
    description='District:',
    placeholder='Select district',
    ensure_option=True
)

# Create an empty soil color widget initially
soil_color_widget = widgets.Combobox(
    description='Soil Color:',
    placeholder='Select soil color',
    ensure_option=True
)

# Checkbox for lab test
lab_test_checkbox = widgets.Checkbox(
    value=False,
    description='Do you have a lab test for soil?',
    disabled=False
)

# Additional parameter widgets
nitrogen_widget = widgets.Combobox(
    description='Nitrogen:',
    placeholder='Select nitrogen value',
    ensure_option=True,
    disabled=True
)

phosphorus_widget = widgets.Combobox(
    description='Phosphorus:',
    placeholder='Select phosphorus value',
    ensure_option=True,
    disabled=True
)

potassium_widget = widgets.Combobox(
    description='Potassium:',
    placeholder='Select potassium value',
    ensure_option=True,
    disabled=True
)

ph_widget = widgets.Combobox(
    description='pH:',
    placeholder='Select pH value',
    ensure_option=True,
    disabled=True
)

rainfall_widget = widgets.Combobox(
    description='Rainfall:',
    placeholder='Select rainfall value',
    ensure_option=True,
    disabled=True
)

temperature_widget = widgets.Combobox(
    description='Temperature:',
    placeholder='Select temperature value',
    ensure_option=True,
    disabled=True
)

recommend_widget = widgets.Output()

# Define the observer function to update options based on selected district
def update_soil_color_options(change):
    district = district_widget.value
    if district:
        soil_colors = dataset[dataset['District_Name'] == district]['Soil_color'].unique()
        soil_color_widget.options = tuple(soil_colors)
    else:
        soil_color_widget.options = ()

# Define the function to toggle additional parameter widgets based on lab test checkbox
def toggle_additional_parameters(change):
    if change.new:
        # Enable additional parameter widgets
        for widget in [nitrogen_widget, phosphorus_widget, potassium_widget, ph_widget, rainfall_widget, temperature_widget]:
            widget.disabled = False
    else:
        # Disable additional parameter widgets
        for widget in [nitrogen_widget, phosphorus_widget, potassium_widget, ph_widget, rainfall_widget, temperature_widget]:
            widget.disabled = True

# Define the function to recommend crop and fertilizer
def recommend_crop(button):
    district = district_widget.value
    soil_color = soil_color_widget.value
    if district and soil_color:
        # Filter dataset based on selected district and soil color
        subset = dataset[(dataset['District_Name'] == district) & (dataset['Soil_color'] == soil_color)]
        if not subset.empty:
            # If lab test is not available or not selected, show average parameters and recommend crop and fertilizer
            if not lab_test_checkbox.value:
                with recommend_widget:
                    recommend_widget.clear_output()
                    print("Average Parameters for Selected District and Soil Color:")
                    print("----------------------------------------------------------")
                    print("Nitrogen:", round(subset['Nitrogen'].mean(), 2))
                    print("Phosphorus:", round(subset['Phosphorus'].mean(), 2))
                    print("Potassium:", round(subset['Potassium'].mean(), 2))
                    print("pH:", round(subset['pH'].mean(), 2))
                    print("Rainfall:", round(subset['Rainfall'].mean(), 2))
                    print("Temperature:", round(subset['Temperature'].mean(), 2))
                    print("\nRecommended Crop:", subset['Crop'].iloc[0])
                    print("Recommended Fertilizer:", subset['Fertilizer'].iloc[0])
                    print("Link:", subset['Link'].iloc[0])
            else:
                # Prepare input data for prediction
                input_data = subset.drop(columns=['Crop', 'Fertilizer', 'Link'])
                target = subset['Crop']
                
                # Perform one-hot encoding
                encoder = OneHotEncoder(handle_unknown='ignore')
                X_encoded = encoder.fit_transform(input_data)
                
                # Split dataset into training and testing sets
                X_train, X_test, y_train, y_test = train_test_split(X_encoded, target, test_size=0.2, random_state=42)
                
                # Train the model
                model_crop = RandomForestClassifier(n_estimators=100, random_state=42)
                model_crop.fit(X_train, y_train)

                # Make predictions
                predicted_crop = model_crop.predict(X_test)

                # Evaluate accuracy
                accuracy = accuracy_score(y_test, predicted_crop)
                with recommend_widget:
                    recommend_widget.clear_output()
                    print("Recommended Crop:", predicted_crop[0])
                    print("Recommended Fertilizer:", subset['Fertilizer'].iloc[0])
                    print("Link:", subset['Link'].iloc[0])
                    print("Accuracy of the model:", round(accuracy, 2))
        else:
            with recommend_widget:
                recommend_widget.clear_output()
                print("No data available for the selected district and soil color.")
    else:
        with recommend_widget:
            recommend_widget.clear_output()
            print("Please select both district and soil color.")

# Add observers to widgets
district_widget.observe(update_soil_color_options, names='value')
lab_test_checkbox.observe(toggle_additional_parameters, names='value')

# Create button widget
button = widgets.Button(description="Recommend")
button.on_click(recommend_crop)

# Arrange widgets
lab_test_widgets = [nitrogen_widget, phosphorus_widget, potassium_widget, ph_widget, rainfall_widget, temperature_widget]
all_widgets = [district_widget, soil_color_widget, lab_test_checkbox] + lab_test_widgets + [button, recommend_widget]

# Display the widgets
widgets.VBox(all_widgets)
